In [1]:
import numpy as np
import pandas as pd

# Load data

In [2]:
kwargs = {"sep": "\t", "low_memory": False}
df = pd.read_csv("../adzrago22smoking/data/21600-0001-Data.tsv", **kwargs)
# ds5 = pd.read_csv("../adzrago22smoking/data/21600-0005-Data.tsv", **kwargs)
# ds8 = pd.read_csv("../adzrago22smoking/data/21600-0008-Data.tsv", **kwargs)
# ds22 = pd.read_csv("../adzrago22smoking/data/21600-0022-Data.tsv", **kwargs)

In [3]:
df.describe()

,AID,IMONTH,IDAY,IYEAR,SCH_YR,BIO_SEX,SMP01,SMP03,H1GI1M,H1GI1Y,...,H1WS4G,H1WS5G,H1WS6G,H1WS7G,H1WS8G,H1WS9G,H1WS10G,H1WS11G,H1WS12G,H1WS13
count,6.504000e+03,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,...,6504.0,6504.0,6504.0,6504.0,6504.0,6504.0,6504.0,6504.0,6504.0,6504.000000
mean,9.417178e+07,6.577645,15.735855,94.999846,0.354397,1.516759,0.933579,0.079951,6.604090,78.970941,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1.038130
std,6.550036e+06,1.403560,8.708040,0.012400,0.483165,0.502825,0.249035,0.271238,3.905455,1.809704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.706785
min,5.710027e+07,1.000000,1.000000,94.000000,0.000000,1.000000,0.000000,0.000000,1.000000,74.000000,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,0.000000
25%,9.257287e+07,5.000000,8.000000,95.000000,0.000000,1.000000,1.000000,0.000000,4.000000,78.000000,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1.000000
50%,9.471444e+07,6.000000,15.000000,95.000000,0.000000,2.000000,1.000000,0.000000,7.000000,79.000000,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1.000000
75%,9.757533e+07,8.000000,23.000000,95.000000,1.000000,2.000000,1.000000,0.000000,9.000000,80.000000,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,1.000000
max,9.971998e+07,12.000000,31.000000,95.000000,6.000000,6.000000,1.000000,1.000000,96.000000,96.000000,...,7.0,97.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,9.000000


# Map data

* Maybe they used different column
* Read the paper
* Multiple responces

In [4]:
column_map = {
    ## Filtering
    # 6 - refused
    "BIO_SEX": "Gender",
    # " " - missing
    "S1": "Age",
    # 6 - refused, 8 - don't know
    "H1GI18": "[If SCHOOL YEAR:] Are you presently in school? [If SUMMER:] Were you in school during this past school year?",
    ## Aggressive delinquency
    # 996 - refused, 997 - legitimate skip, 998 - don't know, 999 - not applicable
    "H1FV13": "During the past 12 months, how many times were you in a physical fight in which you were injured and had to be treated by a doctor or nurse?",
    # 6 - refused, 8 - don't know, 9 - not applicable
    "H1FV7": "During the past 12 months, how often did you pulled a knife or gun on someone.",
    "H1FV8": "During the past 12 months, how often did you shot or stabbed someone.",
    "H1FV9": "During the past 30 days, on how many days did you carry a weapon-such as a gun, knife, or club-to school?",
    ## Non-Aggressive delinquency
    "H1DS3": "In the past 12 months, how often did you lie to your parents or guardians about where you had been or whom you were with?",
    "H1DS4": "How often did you take something from a store without paying for it?",
    "H1DS13": "How often did you steal something worth less than $50?",
    "H1DS9": "In the past 12 months, how often did you steal something worth more than $50?",
    ## Attachment to parents
    # 6 - refused, 7 - legitimate skip, 8 - don't know
    # "H1WP9": "How close do you feel to your biological mother?",
    "H1WP9": "How close do you feel to your mother?",
    # "H1NF14": "How close do you feel to your biological father?",
    "H1WP13": "How close do you feel to your father?",
    # 7 - legitimate skip, 9 - multiple response
    "S16": "How much do you think mother cares about you?",
    "S22": "How much do you think father cares about you?",
    ## Attachment to school
    # 9 - multiple response
    "S62B": "How strongly do you agree or disagree with following statement? I feel close to people at this school.",
    "S62I": "How strongly do you agree or disagree with following statement? I am happy to be at this school.",
    "S62L": "How strongly do you agree or disagree with following statement? The teachers at this school treat students fairly.",
    "S62E": "How strongly do you agree or disagree with following statement? I feel like I am part of this school.",
    ## Attachment to delinquent peers
    # 6 - refused, 8 - don't know, 9 - not applicable
    "H1TO9": "Of your 3 best friends, how many smoke at least 1 cigarette a day?",
    "H1TO29": "Of your 3 best friends, how many drink alcohol at least once a month?",
    "H1TO33": "Of your 3 best friends, how many use marijuana at least once a month?",
    ## Parental monitoring
    # 6 - refused, 7 - legitimate skip, 8 - don't know, 9 - not applicable
    "H1WP1": "Do your parents let you decide about the time you must be home on weekend nights?",
    "H1WP6": "Do your parents let you decide about what time you go to bed on week nights?",
    "H1WP2": "Do your parents let you decide about the people you hang around with?",
    "H1WP3": "Do your parents let you decide about what you wear?",
    "H1WP7": "Do your parents let you decide about what you eat?",
    "H1WP4": "Do your parents let you decide about how much television you watch?",
    "H1WP5": "Do your parents let you decide about which television programs you watch?",
    ## Academic performance
    # 7 - legitimate skip, 8 - I don't know, 9 - multiple response
    "S10A": "Your grades in English/Language Arts",
    "S10B": "Your grades in Mathematics",
    "S10C": "Your grades in History/Social Studies",
    "S10D": "Your grades in Science",
}

In [5]:
df = df[["AID"] + list(column_map.keys())]
df = df.set_index("AID")


def rename(df, add_code=True):
    if add_code:
        m = {k: f"{k} -> {v}" for k, v in column_map.items()}
    else:
        m = column_map
    return df.rename(m, axis=1)

# Filter data

The present study used the public-use data of wave1 ADD Health, which included the sample of 6504 adolescents. We excluded 128 
participants who either refused to answer or were not enrolled in school at the year of the survey, because one of our research interests 
was to examine how young students’ social bonds with their school protected them from delinquency. In this analysis, cases with 
missing values were dropped. With the exclusion of cases with missing data, the final sample consists of 2087 participants. This size of 
the sample provides abundant statistical power for the current analysis. Sensitivity examination confirmed there was no significant 

## Exclude 128 ppl

We excluded 128 
participants who either refused to answer or were not enrolled in school at the year of the survey, because one of our research interests 
was to examine how young students’ social bonds with their school protected them from delinquency. 

In [6]:
df["H1GI18"].value_counts()

1    6376
0     125
8       2
6       1
Name: H1GI18, dtype: int64

In [7]:
df = df[df["H1GI18"] == 1]

## Remove missing

### Remove empty

In [8]:
df.dtypes

BIO_SEX     int64
S1         object
H1GI18      int64
H1FV13     object
H1FV7       int64
H1FV8       int64
H1FV9       int64
H1DS3       int64
H1DS4       int64
H1DS13      int64
H1DS9       int64
H1WP9       int64
H1WP13      int64
S16        object
S22        object
S62B       object
S62I       object
S62L       object
S62E       object
H1TO9      object
H1TO29      int64
H1TO33      int64
H1WP1       int64
H1WP6       int64
H1WP2       int64
H1WP3       int64
H1WP7       int64
H1WP4       int64
H1WP5       int64
S10A       object
S10B       object
S10C       object
S10D       object
dtype: object

Missing values are spaces or empty.

In [9]:
df.select_dtypes("object").apply(lambda col: sorted(col.unique()))

S1              [ , 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
H1FV13    [ , 0, 1, 11, 12, 122, 14, 19, 2, 20, 21, 22, ...
S16                                [ , 1, 2, 3, 4, 5, 7, 9]
S22                                [ , 1, 2, 3, 4, 5, 7, 9]
S62B                                  [ , 1, 2, 3, 4, 5, 9]
S62I                                  [ , 1, 2, 3, 4, 5, 9]
S62L                                  [ , 1, 2, 3, 4, 5, 9]
S62E                                  [ , 1, 2, 3, 4, 5, 9]
H1TO9                              [ , 0, 1, 2, 3, 6, 8, 9]
S10A                            [ , 1, 2, 3, 4, 5, 7, 8, 9]
S10B                            [ , 1, 2, 3, 4, 5, 7, 8, 9]
S10C                            [ , 1, 2, 3, 4, 5, 7, 8, 9]
S10D                               [ , 1, 2, 3, 4, 5, 7, 8]
dtype: object

In [10]:
df = df.apply(lambda col: pd.to_numeric(col, errors="coerce"))

In [11]:
df

,BIO_SEX,S1,H1GI18,H1FV13,H1FV7,H1FV8,H1FV9,H1DS3,H1DS4,H1DS13,...,H1WP6,H1WP2,H1WP3,H1WP7,H1WP4,H1WP5,S10A,S10B,S10C,S10D
AID,,,,,,,,,,,,,,,,,,,,,
57101310,2,17.0,1,0.0,0,0,0,1,0,0,...,1,1,1,1,1,1,3.0,3.0,3.0,4.0
57103171,1,15.0,1,997.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.0,1.0,5.0,4.0
57103869,1,NaN,1,3.0,1,0,0,1,0,0,...,0,1,0,0,0,0,NaN,NaN,NaN,NaN
57104553,2,18.0,1,0.0,0,0,0,3,1,1,...,1,1,1,1,1,1,2.0,1.0,3.0,2.0
57104649,1,12.0,1,1.0,1,0,0,0,0,0,...,0,1,1,1,1,1,2.0,1.0,2.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99719930,2,12.0,1,0.0,0,0,0,1,0,0,...,0,1,1,1,1,0,1.0,1.0,1.0,1.0
99719939,1,13.0,1,997.0,0,0,0,0,0,0,...,0,0,1,1,0,0,2.0,2.0,2.0,2.0
99719970,1,13.0,1,0.0,2,2,1,1,0,0,...,0,1,1,0,1,1,4.0,4.0,4.0,4.0


In [12]:
df = df.dropna()
df.shape

(3869, 33)

In [13]:
df = df.dropna()
df = df.astype(int)
df.shape

(3869, 33)

### Remove other missing values

In this analysis, cases with 
missing values were dropped. With the exclusion of cases with missing data, the final sample consists of 2087 participants. 

In [14]:
df.apply(lambda col: sorted(col.unique()))

BIO_SEX                                               [1, 2]
S1                  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
H1GI18                                                   [1]
H1FV13     [0, 1, 2, 3, 4, 5, 7, 8, 11, 12, 19, 20, 365, ...
H1FV7                                     [0, 1, 2, 6, 8, 9]
H1FV8                                     [0, 1, 2, 6, 8, 9]
H1FV9                               [0, 1, 2, 3, 4, 6, 8, 9]
H1DS3                                  [0, 1, 2, 3, 6, 8, 9]
H1DS4                                  [0, 1, 2, 3, 6, 8, 9]
H1DS13                                 [0, 1, 2, 3, 6, 8, 9]
H1DS9                                  [0, 1, 2, 3, 6, 8, 9]
H1WP9                               [1, 2, 3, 4, 5, 6, 7, 8]
H1WP13                                 [1, 2, 3, 4, 5, 6, 7]
S16                                    [1, 2, 3, 4, 5, 7, 9]
S22                                    [1, 2, 3, 4, 5, 7, 9]
S62B                                      [1, 2, 3, 4, 5, 9]
S62I                    

In [15]:
df.shape

(3869, 33)

In [16]:
cols_where_above_6_is_invalid = [
    "H1FV7",
    "H1FV8",
    "H1FV9",
    "H1DS3",
    "H1DS4",
    "H1DS13",
    "H1DS9",
    "H1WP9",
    "H1WP13",
    "S16",
    "S22",
    "S62B",
    "S62I",
    "S62L",
    "S62E",
    "H1TO9",
    "H1TO29",
    "H1TO33",
    "H1WP1",
    "H1WP6",
    "H1WP2",
    "H1WP3",
    "H1WP7",
    "H1WP4",
    "H1WP5",
    "S62B",
    "S62I",
    "S62L",
    "S62E",
]
for col in cols_where_above_6_is_invalid:
    print(
        f"Remove {df[df[col] >= 6].shape[0]: <5} values for {col: <6} -> {column_map[col]}"
    )

for col in cols_where_above_6_is_invalid:
    df = df[df[col] < 6]

Remove 9     values for H1FV7  -> During the past 12 months, how often did you pulled a knife or gun on someone.
Remove 8     values for H1FV8  -> During the past 12 months, how often did you shot or stabbed someone.
Remove 10    values for H1FV9  -> During the past 30 days, on how many days did you carry a weapon-such as a gun, knife, or club-to school?
Remove 15    values for H1DS3  -> In the past 12 months, how often did you lie to your parents or guardians about where you had been or whom you were with?
Remove 14    values for H1DS4  -> How often did you take something from a store without paying for it?
Remove 15    values for H1DS13 -> How often did you steal something worth less than $50?
Remove 12    values for H1DS9  -> In the past 12 months, how often did you steal something worth more than $50?
Remove 170   values for H1WP9  -> How close do you feel to your mother?
Remove 1052  values for H1WP13 -> How close do you feel to your father?
Remove 230   values for S16    -> How m

In [17]:
df.shape

(2424, 33)

In [18]:
df[df["H1FV13"] > 996].shape

(1225, 33)

In [19]:
df = df[df["H1FV13"] <= 996]
df.shape

(1199, 33)

In [20]:
df.describe()

,BIO_SEX,S1,H1GI18,H1FV13,H1FV7,H1FV8,H1FV9,H1DS3,H1DS4,H1DS13,...,H1WP6,H1WP2,H1WP3,H1WP7,H1WP4,H1WP5,S10A,S10B,S10C,S10D
count,1199.0000,1199.000000,1199.0,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,...,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000
mean,1.3995,14.980817,1.0,2.977481,0.066722,0.025855,0.178482,1.040867,0.412844,0.356964,...,0.674729,0.879900,0.918265,0.819850,0.842369,0.788991,2.796497,2.923269,2.932444,2.908257
std,0.4900,1.676350,0.0,50.866239,0.298385,0.192077,0.700468,1.105079,0.813411,0.770501,...,0.468672,0.325214,0.274075,0.384473,0.364547,0.408195,1.937320,1.932272,2.049375,1.985740
min,1.0000,11.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,1.0000,14.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.0000,15.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000
75%,2.0000,16.000000,1.0,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,4.000000,4.000000,4.000000
max,2.0000,19.000000,1.0,996.000000,2.000000,2.000000,4.000000,3.000000,3.000000,3.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000,9.000000,8.000000,8.000000


In [21]:
Break here

SyntaxError: invalid syntax (2904357748.py, line 1)

In [ ]:
df["H1WP9"]

In [ ]:
df["BIO_SEX"] = df["BIO_SEX"].map({1: "Male", 2: "Female"})

In [ ]:
df

In [ ]:
rename(df, False).groupby("Gender").mean().T[["Male", "Female"]]

In [ ]:
paper_values = [
    [0.59, 0.26, 0.76],
    [0.08, 0.03, 0.94],
    [0.04, 0.01, 0.92],
    [0.17, 0.06, 0.81],
    [0.90, 0.94, 0.68],
    [0.42, 0.30, 0.89],
    [0.08, 0.04, 0.80],
    [0.36, 0.24, 0.88],
    [4.73, 4.59, 0.10],
    [4.98, 4.92, 0.01],
    [5.05, 5.00, 0.97],
    [5.33, 5.39, 0.97],
    [3.83, 3.75, 0.79],
    [3.96, 3.93, 0.82],
    [3.81, 3.71, 0.81],
    [3.57, 3.50, 0.61],
    [0.86, 0.74, 0.82],
    [1.22, 1.10, 0.86],
    [0.67, 0.57, 0.85],
    [0.55, 0.66, 0.78],
    [0.08, 0.08, 0.86],
    [0.02, 0.04, 0.90],
    [0.13, 0.13, 0.88],
    [0.18, 0.18, 0.86],
    [0.30, 0.29, 0.83],
    [0.13, 0.13, 0.86],
    [2.74, 3.08, 0.81],
    [2.75, 2.86, 0.73],
    [2.90, 3.03, 0.80],
    [2.84, 2.99, 0.79],
]
original_numbers = pd.DataFrame(
    paper_values,
    columns=["Male", "Female", "?"],
    index=[column_map[c] for c in df.columns if c not in ["S1", "BIO_SEX", "H1GI18"]],
)

In [ ]:
(
    rename(df, False).groupby("Gender").mean().T[["Male", "Female"]] - original_numbers
).abs()